Osnabrück University - Computer Vision (Winter Term 2022/23) - Prof. Dr.-Ing. G. Heidemann, Ulf Krumnack

# Exercise Sheet 05: Segmentation 2

## Introduction

This week's sheet should be solved and handed in before end of **Sunday, December 11, 2022**. If you need help (and Google and other resources were not enough), feel free to use the Stud.IP forum. Please upload your results to your group's Stud.IP folder.

## Assignment 0: Math recap (Periodic functions) [0 Points]

This exercise is supposed to be very easy, does not give any points, and is voluntary. There will be a similar exercise on every sheet. It is intended to revise some basic mathematical notions that are assumed throughout this class and to allow you to check if you are comfortable with them. Usually you should have no problem to answer these questions offhand, but if you feel unsure, this is a good time to look them up again. You are always welcome to discuss questions with the tutors or in the practice session. Also, if you have a (math) topic you would like to recap, please let us know.

**a)** What are periodic functions? Can you provide a definition?

A periodic function is a function that repeats it values after a given interval, i.e. considering a real function $f:\mathbb{R}\to\mathbb{R}$, this function will be considered periodic, with period length $\lambda\in\mathbb{R}$, if
$$f(x+\lambda) = f(x)\qquad\text{for all $x\in\mathbb{R}$}$$

**b)** What are *amplitude*, *frequency*, *wave length*, and *phase* of a sine function? How can you change these properties?

The amplitude of a periodic function is a measure of its change over a single period. For the standard sine curve $x\mapsto \sin(x)$, the *peak amplitude*, i.e. the height of the highest peak, is $1$ while the *peak-to-peak amplitude*, i.e. the difference between the hightes peak and the lowest through, is $2$. The *wavelenght* is just the period of the sine function, i.e. $\lambda = 2\pi$, while the frequency is its inverse $f=\frac{1}{2\pi}$. One can scale the function value to change the amplitude and the argument to change the frequency/wavelength. So 
$a\cdot\sin(\frac{2\pi}{w}x)$ will give curve with amplitude $a$ and wavelength $w$.

**c)** How are sine and cosine defined for complex arguments? In what sense does this generalize the real case?

A standard way to introduce sine and cosine for complex arguments is based on the fact that the real sine and cosine can be introduced as power series
$$\sin(x) = \sum _{n=0}^{\infty }{\frac {(-1)^{n}}{(2n+1)!}}x^{2n+1}
\qquad\text{and}\qquad
\cos(x) = \sum _{n=0}^{\infty }{\frac {(-1)^{n}}{(2n)!}}x^{2n}$$
This definition can be directly applied to complex arguments. Many of the known trigonometric properties generalize to this case. For purely real arguments, this definition simplifies to the real case.

## Assignment 1: Scale space (5 points)

**a)** What is a scale space? How does it relate to an image pyramid? How is it computed?

For some background on scale spaces, you may have a look at the entry on [Scale space](http://kth.diva-portal.org/smash/get/diva2:441147/FULLTEXT01.pdf) in the Encyclopedia  of  Computer Science and Engineering.

**b**) Explain the figure depicted on CV-07 slide 37. How are the zero crossings obtained? Why do they tend to form loops? How can the depicted information be used for segmentation?

**c)** Implement the computation of a scale space. Also include code for highlighting the zero crossings at different scales to produce a visualization similar to the figure on CV-07 slide 37.

In [3]:
import imageio as imageio
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage
from skimage.filters import sobel

# load the image
img = imageio.imread('images/NewspaperRock.png') / 255.

### BEGIN SOLUTION
# row in the image
row = 150

# the maximal scale in the scale space
max_scale = 40

# scale space resolution
resolution = 400

def compute_scalespace(img, max_scale=2, resolution=200):
    scales = np.linspace(0, max_scale, resolution)

    scalespace = np.zeros((resolution, ) + img.shape, dtype=img.dtype)
    for idx, scale in enumerate(scales):
        scalespace[idx] = ndimage.gaussian_filter(img, sigma=[scale,scale], mode='nearest')
    return scales, scalespace

scales, scalespace = compute_scalespace(img, max_scale=max_scale, resolution=resolution)

scale_img = scalespace[:,row,:].copy()
scale_sobel = sobel(scale_img, axis=1)
crossings = np.zeros_like(scale_img)
crossings[:, :-1] = (scale_sobel[:, 1:] * scale_sobel[:, :-1])
scale_img[crossings<0] = 0

plt.figure()
plt.gray()
plt.imshow(scale_img, vmin=0, vmax=1, origin='lower')
plt.show()
### END SOLUTION

TypeError: sobel() got an unexpected keyword argument 'axis'

## Assignment 2: Texture Segmentation (5 points)

**a)** What is texture? Try to define it in your own words. Can there be a standard definition? What problems do you expect for texture based segmentation? 

Texture refers to a common property in the distribution of gray values or color in a region. Two regions have the same texture, if they agree in that property. That is not a hard definition, but rather a description of the general idea. It can be fleshed out and made more precise by providing a formal specification of the properties, which can be based on different grounds, e.g. based on structural, stochastic, or spectral aspects. However, the suitability of a definition depends on the context: what is texture in one context can be content in another one. Hence one should not expect a general definition that fits for all applications.

Texture is usually not defined for individual pixels but for larger structures. When applied as a homogeneity criterion for region based segmentation, one has to take larger neighborhoods into account. As the segment boundaries are not known in advance (otherwise segmentation would be unnecessary), for boundary pixels such an approach will consider values from multiple segments and hence lead to inconclusive results for such pixels. It is also unsuitable to discover small segments.

**b)** What is a co-occurence matrix? How can it be used to characterize texture?

The co-occurence matrix represents the correlation between pixels. Although the computation of co-occurence values can be defined quite generally, one usually only considers correlations over small distances, i.e., considering only neighboring pixels. If the correlation over larger distances is to be detected, one usually applies multi-scale strategies, i.e. resize the image and then apply the neighbor-pixel version. The matrix then holds the number of gray value combinations of such pixels for all pairs of gray values between for a selected region of the image. From that matrix, some more compact features, like energy, contrast, entropy, etc. can be computed that allow to characterize different textures.


**c)** Implement a function to compute the co-occurence matrix of an image (patch). Apply it and compare your results to (CV-07 slide 54).

In [ ]:
%matplotlib inline
import numpy as np
from scipy import misc
import matplotlib.pyplot as plt
import imageio.v2 as imageio

img = imageio.imread('images/mermaid.png')#, mode='L')

def get_patch(img, x, y, size=40):
    """
    Extract a rectangular patch from an image and mark it in the original image.
    
    Args:
        img (nndarray): Input image.
        x (uint): X-coordinate.
        y (uint): Y-coordinate.
        size (uint): Size of the patch.
        
    Returns:
        result: The extracted patch.
    """
    result = img[x:x+size,y:y+size].copy()
    img[x:x+size, [y,y+1,y+size,y+size+1]] = 0
    img[[x,x+1,x+size,x+size+1], y:y+size] = 0
    return result

patches = []
patches.append(get_patch(img, 50,130))
patches.append(get_patch(img, 110,80))
patches.append(get_patch(img, 260,340))
patches.append(get_patch(img, 310,110))
patches.append(get_patch(img, 100,440))


def cooccurrence(img, dx=1, dy=1):
    """
    Compute a co-occurence matrix for the given image.
    
    Args:
        img          the grayscale image (uint8)
        dx,dy        the offset between the two reference points

    Returns:
        matrix       the co-occurence matrix
    """
    matrix = np.zeros((256, 256))
    # BEGIN SOLUTION
    for g1, g2 in np.ndindex(matrix.shape):
        matrix[g1, g2] = np.sum(
            (img[:img.shape[0] - dy, :img.shape[1] - dx] == g1) & (img[dy:, dx:] == g2)
        ) 
    matrix /= img.size 
    return matrix


# Alternative solution:
def cooccurrence2(img, dx=1, dy=1):
    """
    Compute a co-occurence matrix for the given image.
    
    Args:
        img          the grayscale image (uint8)
        dx,dy        the offset between the two reference points

    Returns:
        matrix       the co-occurence matrix
    """
    matrix = np.zeros((256,256))
    for r,c in np.ndindex((img.shape[0] - dy, img.shape[1] - dx)):
        matrix[img[r, c], img[r + dy, c + dx]] += 1
    matrix /= img.size
    
    # END SOLUTION
    return matrix


plt.figure(figsize=(8, 8))
plt.gray()
plt.imshow(img)
plt.show()


plt.figure(figsize=(8, 8))
i = 0
for p in patches:
    plt.subplot(len(patches),3,i+1); plt.axis('off'); plt.imshow(p)
    # For visualization one may apply some extra me, e.g., logarithmization or binarization
    #plt.subplot(len(patches),3,i+2); plt.imshow(np.log(1 + cooccurrence2(p, 0, 1)), interpolation='none')
    plt.subplot(len(patches),3,i+2); plt.imshow(cooccurrence2(p, 0, 1), interpolation='none')
    plt.subplot(len(patches),3,i+3); plt.imshow(cooccurrence2(p,1,0)>0, interpolation='none')
    i += 3
plt.show()

## Assignment 3: Edge-based segmentation  (5 points)

### a) Gradients
What is the gradient of a pixel? How do we calculate the first, how the second derivative of an image?  

The gradient of a pixel is given by the difference in contrast to its neighboring pixels (4- or 8-neighborhood). The gradient points into the direction with highest divergence. We can imagine an image as a function consisting of two variables (x- and y-axes) and its color shading in each pixel as the outcome. The whole image presents a landscape of valleys and hills in respect to it shading and coloring. A sobel-filtered image presents the first derivative of each pixel while the laplace-filter creates the second derivative. 

### b) Edge linking

Describe in your own words the idea of edge linking. What is the goal? Why does it not necessarily yield closed
edge contours?

After initial segmentation, we try to link loose edges.
* at the start all pixels are marked as unprocessed
* make random pixel a start pixel
* from the start pixel, look perpendicular to the gradient of the start pixel if there are other pixels with similar gradient
* if yes and unprocessed we add them to the edge and make them the new start pixel
* if none can be found we initalize another random start pixel

Why does it not necessarily yield closed edge contours?
* edge linking only considers unprocessed pixels within a certain proximity
* it just searches within the direction of the gradient

### c) Zero crossings

Explain what zero crossings are. Why does the detection of zero crossings always lead to closed contours?

A zero crossing is a point where a function switches signs.

Why does the detection of zero crossings always lead to closed contours?
* intuitive argument: imagine the 2D-function defines a 3D landscape, with function values giving the altitude of the landscape at a given coordinate. Then positive values will be above sea level, while negative values will be below sea level. A zero crossing corresponds to the transition between water and land and this boundary line obviously is continous.

### d) Zero crossings (implementation)

Provide an implementation of the zero crossing procedure described in (CV-07 slide 71). To get sensible results you should smooth the image before applying the Laplacian filter, e.g. using the Laplacian of a Gaussian (you may use buildin functions for the filterings steps).

In [ ]:
from skimage import filters
from skimage.feature import canny
from skimage.color import rgb2gray
from imageio.v3 import imread
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

img = imread('images/swampflower.png').astype(float)
#img = imread('images/peppers.png').astype(float)

if len(img.shape)>2:
    img = rgb2gray(img)
img /= img.max()

# Now compute edges and then zero crossings using the 4-neighborhood and the 8-neighborhood
# BEGIN SOLUTION

# from scipy.ndimage.filters import laplace, gaussian_laplace
# smooth the image
img_smoothed = filters.gaussian(img, sigma=2) # or 2.0, 4.0


# detect edges using a laplacian filter
edges = filters.laplace(img_smoothed)

#hist, _ = np.histogram(img, 256, (0, 255))

# N4 neighborhood
zero_crossings_n4 = (edges[:-1, 1:] * edges[1:, 1:] <= 0) | (edges[1:, :-1] * edges[1:, 1:] <= 0)

# N8 neighborhood
zero_crossings_n8 = (zero_crossings_n4[:, 1:] 
                     | (edges[:-1, 1:-1] * edges[1:, :-2] <= 0) 
                     | (edges[:-1, 1:-1] * edges[1:, 2:] <= 0))

# compute threhshold from histogram and 
# erase zero crossings where gradient magnitude is below threshold
grad_mag = filters.sobel(img_smoothed)[1:,2:]
grad_mag_thresh = filters.threshold_otsu(grad_mag)
print(f"Gradient magnitude threshold: {grad_mag_thresh}")
zero_crossings_n8_thresh = zero_crossings_n8.copy()
zero_crossings_n8_thresh[grad_mag<grad_mag_thresh] = 0
# END SOLUTION

plt.figure(figsize=(12, 24))
plt.gray()

plt.subplot(4,2,1); plt.axis('off'); plt.imshow(img); plt.title('original')
plt.subplot(4,2,2); plt.axis('off'); plt.imshow(edges); plt.title('edges')
plt.subplot(4,2,3); plt.axis('off'); plt.imshow(zero_crossings_n4); plt.title('zero crossings (N4)')
plt.subplot(4,2,4); plt.axis('off'); plt.imshow(zero_crossings_n8); plt.title('zero crossings (N8)' )
plt.subplot(4,2,5); plt.axis('off'); plt.imshow(grad_mag); plt.title('Gradient Magnitude' )
plt.subplot(4,2,6); plt.hist(grad_mag.flatten(), 255, (0, grad_mag.max())); plt.title('Histogram Gradient Magnitude' )
plt.subplot(4,2,7); plt.axis('off'); plt.imshow(zero_crossings_n8_thresh); plt.title('zero crossings (N8) Thresh' )
plt.subplot(4,2,8); plt.axis('off'); plt.imshow(canny(img,sigma=2)); plt.title('Canny')
plt.show()

## Assignment 4: Watershed transform  (5 points)



### a) Watershed transform

Explain in your own words the idea of watershed transform. How do the two different approaches from the lecture work? Why does watershed transform always give a closed contour?



Idea of watershed:
Find the boundaries were water would meet if the whole region was iteratively flooded.

How do the two different approaches from the lecture work?
* rain: assume that it rains on all pixels. Water flows opposite to the gradient.
* flood: ground water level is contionously rising

Why does watershed transform always give a closed contour?
* The image is "continous" in both dimensions there will always be a local maximum, separating two basins.

### b) Implementation

Now implement the watershed transform using the flooding approach (CV-07 slide 76, but note, that the algorithm presented there is somewhat simplified!). Obviously, buildin functions for computing watershed transform are not allowed, but all other functions may be used. In this example we appply the watershed transform to a distance transformed image, so you **do not** have to take the gradient image, but can apply the watershed transform directly.

In [ ]:
import numpy as np
import imageio.v3 as imageio
import matplotlib.pyplot as plt
%matplotlib inline


def watershed(img, step=1):
    """
    Perform watershed transform on a grayscale image.
    
    Args:
        img (ndarray): The grayscale image.
        step (int): The rise of the waterlevel at each step. Default 1.
        
    Returns:
        edges (ndarray): A binary image containing the watersheds.
    """

    NO_LABEL = 0
    WATERSHED = 1
    new_label = 2

    # initialize labels
    label = np.zeros(img.shape, np.uint16)

    # BEGIN SOLUTION
    for level in np.arange(np.ceil(img.min()), np.ceil(img.max()) + 1, step):
        # Remember highest label that was assigned in the last iteration.
        previous_new_label = new_label
        
        # Iterate over all unlabeled pixels below water level
        for r, c in np.argwhere((img <= level) & (label == NO_LABEL)):
            # Get labels of neighbors.
            neighbors = label[max(0, r - 1):min(img.shape[0], r + 2),
                              max(0, c - 1):min(img.shape[1], c + 2)]
            flooded = np.unique(neighbors[neighbors > WATERSHED])
            
            if flooded.size == 0:
                # Pixel is isoloated: invent a new label.
                new_label += 1
                label[r, c] = new_label
            elif flooded.size == 1:
                # Pixel has homogenous neighborhood.
                label[r, c] = flooded[0]
            else:
                # Pixel has inhomogenous neighborhood.
                old = flooded[flooded <  previous_new_label] # Neighbors flooded with previous waterlevels.
                new = flooded[flooded >= previous_new_label] # Neighbors flooded with current waterlevel.
                # If region is flooded for the first time, give all newly flooded neighbors the same label.
                if old.size == 0:
                    label[r, c] = new[0]
                    label[np.isin(label, new[1:])] = label[r, c]
                # If parts of the region were flooded before, propagate old label
                # to newly flooded pixels.
                elif old.size == 1:
                    label[r, c] = old[0]
                    label[np.isin(label, new)] = label[r, c]
                # Two or more independtly flooded regions meet here,
                # hence the pixel is a watershed.
                else:
                    label[r, c] = WATERSHED            
    edges = (label == WATERSHED)
    return edges
    # END SOLUTION


img = imageio.imread('images/dist_circles.png')

plt.gray()
plt.subplot(1,2,1)
plt.axis('off')
plt.imshow(img)

plt.subplot(1,2,2)
plt.axis('off')
plt.imshow(watershed(img))
plt.show()

### c) Application: maze

You can use watershed transform to find your way through a maze. To do so, first apply a distance transform to the maze and the flood the result. The watershed will show you the way through the maze. Explain why this works.
You can use build-in functions instead of your own watershed function.

In [ ]:
import numpy as np
import imageio.v3 as imageio
import matplotlib.pyplot as plt
from scipy.ndimage import distance_transform_edt
%matplotlib inline

img = imageio.imread('images/maze2.png') # 'maze1.png' or 'maze2.png'

dt = distance_transform_edt(img)
#result = img[:, :, np.newaxis].repeat(3, 2)
result = dt[:,:,np.newaxis].repeat(3,2) / 255
# BEGIN SOLUTION
#result[:,:,:] = dt
result[watershed(dt), 1:3] = 0
# END SOLUTION

plt.figure(figsize=(10, 10))
plt.title('Solution')
plt.axis('off')
plt.gray()
plt.imshow(result)
plt.show()

The distance transformed image has its highest values when the points are most distance to the edges.
The watershed transform will mark those local maxima as a way through the maze.